# Hello Python 출력 실습

In [2]:
print("Hello Python")

Hello Python


# If you know Numpy

In [3]:
import numpy as np # 다차원배열에 대한 연산을 빠르게 처리할 수 있게 하는 라이브러리

a = np.zeros((2, 2)) # 배열의 모든 값이 0인 2x2 배열을 생성
b = np.ones((2, 2)) # 배열의 모든 값이 1인 2x2 배열을 생성

np.sum(b, axis=1) # axis=1은 각 행에 대한 합계를 계산


array([ 2.,  2.])

In [4]:
a.shape # a 배열의 행과 열을 출력

(2, 2)

In [5]:
np.reshape(a, (1, 4)) # a 배열의 행과 열을 1, 4로 재구성

array([[ 0.,  0.,  0.,  0.]])

# Repeat in TensorFlow

In [2]:
import tensorflow as tf # Google에서 개발한 기계 학습 엔진
sess = tf.Session() # tensorflow를 CPU/GPU 등의 연산장치에 올려놓는 것

a = tf.zeros((2, 2))
b = tf.ones((2, 2))

sess.run(tf.reduce_sum(b, axis=1)) # 지정된 차원을 따라 엘리먼트들을 더함

array([ 2.,  2.], dtype=float32)

In [7]:
a.get_shape() # Tensor의 크기 확인

TensorShape([Dimension(2), Dimension(2)])

In [8]:
sess.run(tf.reshape(a, (1, 4))) # a 배열의 행과 열을 1, 4로 재구성

array([[ 0.,  0.,  0.,  0.]], dtype=float32)

# sess.run()

In [9]:
sess = tf.Session()
a = np.zeros((2, 2))
ta = tf.zeros((2, 2))

print(a)

[[ 0.  0.]
 [ 0.  0.]]


In [10]:
# TensorFlow first defines a computation graph that has no numerical value until evaluated.
# (very similar concept to Theano)
print(ta)

Tensor("zeros_1:0", shape=(2, 2), dtype=float32)


In [11]:
print(sess.run(ta))

[[ 0.  0.]
 [ 0.  0.]]


# Computation Graph

* In TensorFlow program, it first assembles a **graph**, and uses a session to **execute** ops in the graph.

In [12]:
# Define a computation graph.
# At this time, we cannot see real value.
a = tf.constant(5.0) # 상수선언 -> 5
b = tf.constant(6.0) # 상수선언 -> 6
c = a * b

# Evaluate pre-defined computation graph.
# Graph evaluation must be followed by tf.Session creation.
sess = tf.Session()
print(sess.run(c))

30.0


# TensorFlow Session

* A **Session** object encapsulates the environment in which operation and tensor objects are evaluated
* Both type of the session creation has same behavior

In [13]:
sess = tf.Session()
print(sess.run(c))

with tf.Session() as sess:
    print(sess.run(c))
    # Syntactic sugar for sess.run(c) in the current active session.
    # (Must locate inside of the tf.Session scope)
    print(c.eval()) # eval() -> 실행 가능한 문자열(1+2, 'hel'+'lo')을 입력으로 받아 문자열을 실행한 결과값을 리턴하는 함수

30.0
30.0
30.0


# TensorFlow Variables

* All tensors used previously were **constant** tensors
* To train a model, we need **variable** type tensor to hold and update values

In [14]:
w = tf.Variable(tf.zeros((2, 2)), name="weight")
with tf.Session() as sess:
    print(sess.run(w))
    
# Traceback (most recent call last):
# 
# tensorflow.python.framework.errors_impl.FailedPreconditionError:
# Attempting to use uninitialized value weight

FailedPreconditionError: Attempting to use uninitialized value weight
	 [[Node: _send_weight_0 = _Send[T=DT_FLOAT, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=5051602978415562487, tensor_name="weight:0", _device="/job:localhost/replica:0/task:0/cpu:0"](weight)]]

# Variables need initialization!

In [15]:
w = tf.Variable(tf.random_normal([5, 2], stddev=0.1), name="weight")
# tf.Variable can be initialized from constants or random values

with tf.Session() as sess:
    # tf.Variable needs initialization step.
    sess.run(tf.global_variables_initializer()) # 변수 초기화 명령어
    print(sess.run(w))

[[-0.05101353  0.06409221]
 [-0.01627205 -0.00306728]
 [-0.1312844  -0.09858428]
 [ 0.27273777 -0.12430476]
 [ 0.02140464 -0.08927739]]


# Updating Variables

In [16]:
state = tf.Variable(0, name="counter") # 변수 선언
new_Value = tf.add(state, tf.constant(1)) # state(0)과 1을 더하는 new_value 생성
update = tf.assign(state, new_Value) # 오퍼레이션을 이용해서 변수의 초기값을 설정

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # 변수 초기화
    print(sess.run(state)) # '0' 출력
    for _ in range(3):
        sess.run(update) # state + 1을 실행한 후 state에 저장해줌
        print(sess.run(state))

0
1
2
3


# Fetching Variables

<img src="./picture/fetching_variables.jpg" width="40%"/>

In [17]:
x1 = tf.constant(1) # 상수 선언 -> 1
x2 = tf.constant(2) # 상수 선언 -> 2
x3 = tf.constant(3) # 상수 선언 -> 3
temp = tf.add(x2, x3) # 2 + 3 = 5
mul = tf.multiply(x1, temp) # 1 * 5 = 5

with tf.Session() as sess:
    # Calling sess.run(var) outputs its value.
    # Of course, we can output multiple tensors simultaneously with sess.run([var1, .. ,])
    result1, result2 = sess.run([mul, temp]) # result1 = mul, result2 = temp
    print(result1, result2) # result1, result2 출력

5 5


# TensorFlow Placeholder

* All previous examples have manually defined tensors
    * How can we input external data into TensorFlow?
    * Most simple way is using **tf.placeholder** and **feed_dict**

In [18]:
# Create a placeholder to hold data, tf.placeholder is dummy node to provide entry points for data to computation graph.
a = tf.placeholder(tf.int16) # 다양한 데이터를 입력 받아 처리할 수 있도록 지원하는 오퍼레이션
b = tf.placeholder(tf.int16)

add = tf.add(a, b)
mul = tf.multiply(a, b)

with tf.Session() as sess:
    # A feed_dict is a python dictionary from tf.placeholder to data value
    print(sess.run(add, feed_dict={a:2, b:3})) # a:2, b:3을 입력하고 add 오퍼레이션을 실행
    print(sess.run(mul, feed_dict={a:2, b:3})) # a:2, b:3을 입력하고 multiply 오퍼레이션을 실행

5
6


In [ ]:
# using tf.constant
import tensorflow as tf
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.], [2.]])
product = tf.matmul(matrix1, matrix2) # matrix1, matrix2 배열을 곱한 결과를 product로 반환

with tf.Session() as sess:
    result = sess.run(product) # product 실행 결과를 result로 반환
    print(result)
    
# using placeholder
import numpy as np

matrix1 = tf.placeholder(tf.float32, [1, 2]) # 행과 열이 1, 2인 배열 생성
matrix2 = tf.placeholder(tf.float32, [2, 1]) # 행과 열이 2, 1인 배열 생성
product = tf.matmul(matrix1, matrix2) # matrix1, matrix2 배열을 곱한 결과를 product로 반환

with tf.Session() as sess:
    mv1 = np.array([[3., 3.]])
    mv2 = np.array([[2.], [2.]])
    result = sess.run(product, feed_dict={matrix1: mv1, matrix2: mv2}) # matrix1:mv1, matrix2:mv2로 입력한 후 product 오퍼레이션 실행결과를 reulst로 반환
    print(result)

# Example - MNIST with MLP

In [43]:
import tensorflow as tf

# Import MNIST data
# MNIST -> 28x28 크기의 0~9 사이의 숫자 이미지와 이에 해당하는 레이블로 구성된 데이터베이스
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/",one_hot=True) 

learning_rate = 0.001
max_steps = 15000
batch_size = 128

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape --> 28*28 = 784
y = tf.placeholder(tf.float32, [None, 10]) # 0~9 digits recognition --> 10 classes

def MLP(inputs):
    W_1 = tf.Variable(tf.random_normal([784, 256]))
    b_1 = tf.Variable(tf.zeros([256]))

    W_2 = tf.Variable(tf.random_normal([256, 256]))
    b_2 = tf.Variable(tf.zeros([256]))

    W_out = tf.Variable(tf.random_normal([256, 10]))
    b_out = tf.Variable(tf.zeros([10]))

    h_1 = tf.add(tf.matmul(inputs, W_1), b_1)
    h_1 = tf.nn.relu(h_1)

    h_2 = tf.add(tf.matmul(h_1, W_2), b_2)
    h_2 = tf.nn.relu(h_2)

    out = tf.add(tf.matmul(h_2, W_out), b_out)

    return out

net = MLP(x)

# define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=net))
# tf.reduce_mean --> 엘리먼트들의 평균
# x -> [[1., 1.]
#       [2., 2.]] 일 경우   
# 
# tf.reduce_mean(x) --> 1.5
# tf.reduce_mean(x, 0) --> [1.5, 1.5]
# tf.reduce_mean(x, 1) --> [1., 2.]

# softmax_cross_entropy_with_logits
# softmax + cross entropy를 한번에 계산하는 함수

# softmax
# 뉴런의 출력값을 정규화하는 함수

# cross entropy
# 두 개의 확률 분포 간의 차이를 나타내는 지표
# 차이가 클수록 큰 값이 나오고, 두 개가 같아질 때 최소값이 나옴

opt = tf. train.AdamOptimizer(learning_rate).minimize(loss_op)
# AdamOptimizer --> 현존하는 Optimizer 중에 가장 좋은 Optimizer

# initiailizing  the variables
init_op = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init_op)

#train model
for step in range(max_steps):
    batch_X, batch_Y = mnist.train.next_batch(batch_size)
    # mnist.train,next_batch --> 데이터를 100개씩 랜덤으로 뽑아줌
    _, loss = sess.run([opt, loss_op], feed_dict={x: batch_X, y: batch_Y})
    
    if (step+1) % 1000 == 0:
        print("[{}/{}] loss:{:.3f}".format(step+1, max_steps, loss))
print("Optimization Finished!")

# test model
correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y, 1))
# tf.argmax --> 텐서 내의 지정된 축에서 가장 높은 값의 인덱스를 반환

# calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Train accuracy: {:.3f}".format(sess.run(accuracy, feed_dict={x: mnist.train.images, y: mnist.train.labels})))
print("Test accucary: {:.3f}".format(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})))


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
[1000/15000] loss:36.668
[2000/15000] loss:25.181
[3000/15000] loss:8.980
[4000/15000] loss:3.095
[5000/15000] loss:1.137
[6000/15000] loss:1.322
[7000/15000] loss:0.000
[8000/15000] loss:0.000
[9000/15000] loss:0.161
[10000/15000] loss:0.000
[11000/15000] loss:1.379
[12000/15000] loss:0.000
[13000/15000] loss:0.000
[14000/15000] loss:0.000
[15000/15000] loss:0.497
Optimization Finished!
Train accuracy: 0.997
Test accucary: 0.956


<img src="./picture/example_mnist_with_mlp.jpg" width="70%"/>

# Training a model

* **tf.nn.softmax_cross_entropy_with_logits** is for calculate cross entropy loss between predictions and labels (instead, you can make own loss function)

* **tf.train.####Optimizer** create an optimizer such as SGD, RMSProp, Adam etc..

* **tf.train.####Optimizer.minimize(loss_op)** adds optimization operation to computation graph

* we don't have to be concern about computing gradients and updating variables

# Variable Scope

* Complicated TensorFlow models can have hundreds of variables
    * **tf.variable_scope()** provides simple name-spacing to avoid clashes
    * **tf.get_variable()** vreates/accesses variables from within a variable scope

# tf.variable_scope()

* Variable scope is a simple type of name-spacing that adds **prefixes** to variable names within scope

In [20]:
var1 = tf.Variable([1], name="var")
with tf.variable_scope("foo"): # variable_scope --> op와 variable의 namespace를 생성
    with tf.variable_scope("bar"):
        var2 = tf.Variable([1], name="var")
        var3 = tf.Variable([1], name="var")
        
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: var:0
var2: foo/bar/var:0
var3: foo/bar/var_1:0


# tf.get_variable()

* **tf.Variable** doesn't provide variable reuse
    * Variable reuse is necessary to implement RNN or Recursive NN

In [3]:
var1 = tf.Variable([1], name="var")
with tf.variable_scope("foo"):
    with tf.variable_scope("bar") as scp:
        var2 = tf.Variable([1], name="var")
        scp.reuse_variables() # allow reuse variables
        var3 = tf.Variable([1], name="var")
        
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: var:0
var2: foo/bar/var:0
var3: foo/bar/var_1:0


<img src="./picture/tf.get_variable().jpg" width="80%"/>

In [4]:
var1 = tf.get_variable("var", [1])
with tf.variable_scope("foo"):
    with tf.variable_scope("bar") as scp:
        var2 = tf.get_variable("var", [1])
        scp.reuse_variables()
        var3 = tf.get_variable("var", [1])
        
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: var_1:0
var2: foo/bar/var_2:0
var3: foo/bar/var_2:0


# Parameter sharing

In [29]:
import tensorflow as tf

with tf.variable_scope("foo"):
    with tf.variable_scope("bar") as scp:
        var1 = tf.get_variable("var", [1])
        scp.reuse_variables()
        var2 = tf.get_variable("var", [1])
        
    with tf.variable_scope("bar", reuse=True):
        var3 = tf.get_variable("var", [1])
        
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: foo/bar/var_2:0
var2: foo/bar/var_2:0
var3: foo/bar/var_2:0


# Wrappers

In [19]:
# Another init strategy called He init introduced in PReLUNet
from tensorflow.contrib.layers import variance_scaling_initializer
he_init = variance_scaling_initializer()

def conv(bottom, num_filter, ksize=3, stride=1, padding="SAME", scope=None):
    # To calculate shape of previous(input) layer
    bottom_shape = bottom.get_shape().as_list()[3]
    
    with tf.variable_scope(scope or "conv"):
        W = tf.get_variable("W", [ksize, ksize, bottom_shape, num_filter], initializer=he_init)
        b = tf.get_variable("b", [num_filter], initializer=tf.constant_initializer(0))
        
        x = tf.nn.conv2d(bottom, W, strides=[1, stride, stride, 1], padding=padding)
        x = tf.nn.relu(tf.nn.bias_add(x, b))
        
    return x

In [20]:
def maxpool(bottom, ksize=2, stride=2, padding="SAME", scope=None):
    with tf.variable_scope(scope or "maxpool"):
        pool = tf.nn.max_pool(bottom, ksize=[1, ksize, ksize, 1], strides=[1, stride, stride, 1], padding=padding)
    return pool

In [21]:
def fc(bottom, num_dims, scope=None):
    bottom_shape = bottom.get_shape().as_list()
    # If dim of bottom layer is more than 2, flatten
    # * output of conv has (N, H, W, C) shape
    # * output of FC has (N, C) shape
    if len(bottom_shape) > 2:
        bottom = tf.reshape(bottom, [-1, reduce(lambda x, y: x*y, bottom_shape[1:])])
        bottom_shape = bottom.get_shape().as_list()
        
    with tf.variable_scope(scope or "fc"):
        W = tf.get_variable("W", [bottom_shape[1], num_dims], initializer=he_init)
        b = tf.get_variable("b", [num_dims], initializer=tf.constant_initializer(0))
        
        out = tf.nn.bias_add(tf.matmul(bottom, W), b)
    return out

def fc_relu(bottom, num_dims, scope=None):
    with tf.variable_scope(scope or "fc"):
        out = fc(bottom, num_dims, scope="fc")
        relu = tf.nn.relu(out)
        
        return relu

# All Together

In [22]:
# To hold keeping probability in dropout Feed value between (0, 1.0) in training, feed 1.0 in test time
keep_prob = tf.placeholder(tf.float32, None)

def conv_net(x, keep_prob):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    conv1 = conv(x, 32, 5, scope="conv_1")
    conv1 = maxpool(conv1, scope="maxpool_1")
    conv2 = conv(conv1, 64, 5, scope="conv_2")
    conv2 = maxpool(conv2, scope="maxpool_2")
    
    fc1 = fc_relu(conv2, 1024, scope="fc_1")
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    out = fc(fc1, 10, scope="out")
    return out

# Session config

* By default, session consume **all** the GPU resource, no matter how a model capacity is
    * **With killing all the other prosess!!! :(**
* Try this config

In [5]:
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)

# Layers

In [24]:
import tensorflow as tf

input = ...
with tf.name_scope('conv1_1') as scope:
    kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32, stddev=1e-1), name='weights')
    conv = tf.nn.conv2d(input, kernel, [1, 1, 1, 1], padding='SAME')
    biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32), trainable=True, name='biases')
    bias = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(bias, name=scope)

TypeError: Expected binary or unicode string, got Ellipsis

In [25]:
import tensorflow.contrib.slim as slim

# 1. simple network generation with slim
net = ...
net = slim.conv2d(net, 256, [3, 3], scope='conv3_1')
net = slim.conv2d(net, 256, [3, 3], scope='conv3_2')
net = slim.conv2d(net, 256, [3, 3], scope='conv3_3')
net = slim.max_pool2d(net, [2, 2], scope='pool3')

# 2. cleaner by repeat operation:
net = ...
net = slim.repeat(net, 3, slim,conv2d, 256, [3, 3], scope='conv3')
net = slim.max_pool(net, [2, 2], scope='pool3')

# 3. Verbose way:
x = slim.fully_connected(x, 32, scope='fc/fc_1')
x = slim.fully_connected(x, 64, scope='fc/fc_2')
x = slim.fully_connected(x, 128, scope='fc/fc_3')

# 4. Equivalent, TF-Slim way using slim.stack:
slim.stack(x, slim.fully_connected, [32, 64, 128], scope='fc')

TypeError: Expected binary or unicode string, got Ellipsis

# argscope

* Eliminate boilerplate codes

In [26]:
he_init = slim.variance_scaling_initializer()
xavier_init = slim.xavier_initializer()

with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn = tf.nn.relu, weights_initializer=he_init, weights_regularizer=slim.l2_regularizer(0.0005)):
    with slim.arg_scope([slim.conv2d], stride=1, padding='SAME'):
        net = slim.conv2d(inputs, 64, [11, 11], 4, scope='conv1')
        net = slim.conv2d(net, 256, [5, 5], weights_initializer=xavier_init, scope='conv2')
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc')

NameError: name 'inputs' is not defined

# Losses

* Easy to handle multiple loss and regularize loss

In [2]:
import tensorflow.contrib.slim as slim

# Define the loss functions and get the total loss.
loss1 = slim.losses.softmax_cross_entropy(pred1, label1)
loss2 = slim.losses.mean_squared_error(pred2, label2)

# The following two lines have the same effect:
total_loss = loss1 + loss2
slim.losses.get_total_loss(add_regularization_losses=False)

# If you want to add regularization loss
reg_loss = tf.add_n(slim.losses.get_regularization_losses())
total_loss = loss1 + loss2 + reg_loss

# or
total_loss = slim.losses.get_total_loss()

NameError: name 'pred1' is not defined

# Collection

* Included in tf.(not slim), anyway it's quite useful
    * GLOBAL_VARIABLES
    * TRAINABLE_VARIABLES
    * SUMMARIES
    * MOVING_AVERAGE_VARIABLES
    * REGULARIZATION_LOSSES
    * WEIGHTS
    * BIASES
    * ACTIVATIONS

In [3]:
@add_arg_scope
# Add custom loss to LOSSES collection
def add_loss(loss, loss_collection=ops.GraphKeys.LOSSES):
    if loss_collection:
        ops.add_to_collection(loss_collection, loss)
        
# Easy to get all losses with graph Collection
def get_losses(scope=None, loss_collection=ops.GraphKeys.LOSSES):
    return ops.get_collection(loss_collection, scope)

def get_regularization_losses(scope=None):
    return ops.get_collection(ops.GraphKeys.REGULARIZATION_LOSSES, scope)

# Useful to visualize or debugging
for var in tf.trainable_variables():
    print(var.name)
    
def trainable_variables():
    return ops.get_collection(ops.GraphKeys.TRAINABLE_VARIABLES)

NameError: name 'add_arg_scope' is not defined

# Save/Restore

* My **save** wrapper function in my custom network class

In [4]:
import tensorflow as tf

In [5]:
def save(self, ckpt_dir, global_step=None):
    # Create saver to save a model
    if self.config.get("saver") is None:
        self.config["saver"] = \
            tf.train.Saver(max_to_keep=30)
            
    saver = self.config["saver"]
    sess = self.config["sess"]
    
    dirname = os.path.join(ckpt_dir, self.name)
    
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    # Save a model in dirname directory.
    # global_step is for recording "current step" in filename
    saver.save(sess, dirname, global_step)

* My **restore** wrapper function

In [ ]:
def load_latest_checkpoint(self, ckpt_dir, exclude=None):
    path = tf.train.latest_checkpoint(ckpt_dir)
    if path is None:
        raise AssertionError("No ckpt exists in {0}.".format(ckpt_dir))
        
    print("Load {} save file".format(path))
    self._load(path, exclude)
    
def load_from_path(self, ckpt_path, exclude=None):
    self._load(ckpt_path, exclude)
    
def _load(self, ckpt_path, exclude):
    init_fn = slim.assign_from_checkpoint_fn(ckpt_path, slim.get_variables_to_restore(exclude=exclude), ignore_missing_vars=True)
    init_fn(self.config["sess"])

# VGG-16

In [17]:
# Build network
def vgg_16(inputs, num_classes=1000, is_training=True, dropout_keep_prob=0.5, spatial_squeeze=True, scope='vgg_16'):
    with tf.variable_scope(scope, 'vgg_16', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'
        # Collect outputs for conv2d, fully_connected and max_pool2d
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d], outputs_collections=end_points_collection):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2')
            net = slim.repeat(net, 2, slim.conv2d, 256, [3, 3], scope='conv3')
            net = slim.max_pool2d(net, [2, 2], scope='pool3')
            net = slim.repeat(net, 2, slim.conv2d, 512, [3, 3], scope='conv4')
            net = slim.max_pool2d(net, [2, 2], scope='pool4')
            net = slim.repeat(net, 2, slim.conv2d, 512, [3, 3], scope='conv5')
            net = slim.max_pool2d(net, [2, 2], scope='pool5')
            # Use conv2d instead of fully_connected layers.
            net = slim.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training, scope='dropout6')
            net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training, scope='dropout7')
            net = slim.conv2d(net, num_classes, [1, 1], activation_fn=None, normalizer_fn=None, scope='fc8')
            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            if spatial_squeeze:
                net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                end_points[sc.name + '/fc8'] = net
            return net, end_points
vgg_16.default_image_size = 224

# VGGNet argscope
def vgg_arg_scope(weight_decay=0.0005):
    with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu, weights_regularizer=slim.l2_regularizer(weight_decay), biases_initializer=tf.zeros_initializer):
        with slim.arg_scope([slim.conv2d], padding='SAME') as arg_sc:
            return arg_sc

In [18]:
X = tf.placeholder(tf.float32, [None, 224, 224, 3], name="X")
y = tf.placeholder(tf.int32, [None, 8], name="y")
is_training = tf.placeholder(tf.bool, name="is_training")
with slim.arg_scope(vgg_arg_scope()):
    net, end_pts = vgg_16(X, is_training=is_training, num_classes=1000)
    
with tf.variable_scope("losses"):
    cls_loss = slim.losses.softmax_cross_entropy(net, y)
    reg_loss = tf.add_n(slim.losses.get_regularization_losses())
    loss_op = type_loss + reg_loss
    
with tf.variable_scope("opt"):
    opt = tf.train.AdamOptimizer(0.001).minimize(loss_op)

# Exclude fc8 layer to train last layer from scratch
# To freeze other layers, trainable=False argument is needed in argscope
self.load_from_path(ckpt_path=VGG_PATH, exclude=["vgg_16/fc8"])

TypeError: __init__() got multiple values for argument 'dtype'

# TensorBoard

* Very useful visualization tool
    * Graph visualization, loss/accuracy plot, weight histogram plot...
<img src="./picture/tensorboard.jpg" width="70%" />

* Simple usage
    1. Make a **tf.summary.FileWriter** to write summaries
    2. Create summaries using **tf.summary.###**
    3. Run summaries with **sess.run(...)**
    4. Add summaries to **FileWriter** with **add_summary** API
    5. Run command
        **tensorboard --logdir=## --host=## --port=##**
    6. Default port is 6006

# TensorBoard & slim example

In [11]:
import tensorflow as tf
slim = tf.contrib.slim

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/",one_hot=True)

max_steps = 10000
batch_size = 128
lr = 0.001
keep_prob = 0.5
weight_decay = 0.0004
logs_path = "/tmp/tensorflow_logs/example"

def my_arg_scope(is_training, weight_decay):
    with slim.arg_scope([slim.conv2d], activation_fn=tf.nn.relu, weights_regularizer=slim.l2_regularizer(weight_decay), weights_initializer=slim.variance_scaling_initializer(), biases_initializer=tf.zeros_initializer, stride=1, padding="SAME"):
        with slim.arg_scope([slim.dropout], is_training=is_training) as arg_sc:
            return arg_sc
        
def my_net(x, keep_prob, outputs_collections="my_net"):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    with slim.arg_scope([slim.conv2d, slim.max_pool2d], outputs_collections=outputs_collections):
        net = slim.conv2d(x, 64, [3, 3], scope="conv1")
        net = slim.max_pool2d(net, [2, 2], scope="pool1")
        net = slim.conv2d(net, 128, [3, 3], scope="conv2")
        net = slim.max_pool2d(net, [2, 2], scope="pool2")
        net = slim.conv2d(net, 256, [3, 3], scope="conv3")
        # global average pooling
        net = tf.reduce_mean(net, [1, 2], name="pool3", keep_dims=True)
        net = slim.dropout(net, keep_prob, scope="dropout3")
        net = slim.conv2d(net, 1024, [1, 1], scope="fc4")
        net = slim.dropout(net, keep_prob, scope="dropout4")
        net = slim.conv2d(net, 10, [1, 1], activation_fn=None, scope="fc5")
        
    # Gather variables into end_points
    end_points = \
        slim.utils.convert_collection_to_dict(outputs_collections)
    return tf.reshape(net, [-1, 10]), end_points

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

with slim.arg_scope(my_arg_scope(is_training, weight_decay)):
    net, end_pts = my_net(x, keep_prob)
    pred = slim.softmax(net, scope="prediction")
    
with tf.variable_scope("losses"):
    cls_loss = slim.losses.softmax_cross_entropy(net, y)
    reg_loss = tf.add_n(slim.losses.get_regularization_losses())
    loss_op = cls_loss + reg_loss
    
# To gather grad to visualize
with tf.variable_scope("Adam"):
    opt = tf.train.AdamOptimizer(lr)
    # Op to calculate every variable gradient
    grads = tf.gradients(loss_op, tf.trainable_variables())
    grads = list(grads, tf.trainable_variables())
    # Op to update all variables according to their gradient
    apply_grads = opt.apply_gradients(grads_and_vars=grads)
    
with tf.variable_scope("accuracy"):
    correct_op = tf.equal(tf.argmax(net, 1), tf.argmax(y, 1))
    acc_op = tf.reduce_mean(tf.cast(correct_op, tf.float32))
    
# Create a summary to monitor loss and accuracy
summ_loss = tf.summary.scalar("loss", loss_op)
summ_acc = tf.summary.scalar("accuracy_test", acc_op)

# Create summaries to visualize weights and grads
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var, collections=["my_summ"])
for grad, var in grads:
    tf.summary.histogram(var.name + "/gradient", grad, collections=["my_summ"])
    
summ_wg = tf.summary.merge_all(key="my_summ")

sess = tf.Session()
sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)

for step in range(max_steps):
    batch_X, batch_y = mnist.train.next_batch(batch_size)
    _, loss, plot_loss, plot_wg = sess.run([apply_grads, loss_op, summ_loss, summ_wg], feed_dict={x: batch_X, y: batch_y, is_training: True})
    summary_writer.add_summary(plot_loss, step)
    summary_writer.add_summary(plot_wg, step)
    
    if (step+1) % 100 == 0:
        plot_acc = sess.run(summ_acc, feed_dict={x: mnist.test.images, y: mnist.test.labels, is_training: False})
        summary_writer.add_summary(plot_acc, step)
print("Optimization Finished!")

test_acc = sess.run(acc_op, feed_dict={x: mnist.test.images, y: mnist.test.labels, is_training: False})
print("Test accuracy: {:.3f}".format(test_acc))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


TypeError: __init__() got multiple values for argument 'dtype'